In [4]:
# Instalamos el driver de cassandra
# sudo pip install cassandra-driver

In [10]:
# Vamos a importar
from cassandra.cluster import Cluster
# Vamos a conectarno a la base de datos:

## Iniciareos el contenedor de Docker con la iagen de cassandra:
'''
docker pull cassandra

Cluster: Para esta práctica solo usaremos un nodo
docker run --name prac2 -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra

Conexión:
docker exec -it prac2 bash

Información del Nodo:
nodetool status

'''
# Indicación de nuestra imagen

cluster = Cluster(contact_points=['0.0.0.0'], port=9042)

# Generar la conexión
session = cluster.connect()

In [11]:
'''
Creación del KEYSPACE:  que vamos a llamar peliculas
y le vamos a poner una estrategia de replicación de tipo 
SimpleStrategy y con un factor de replicación igual a 1.
'''
session.execute("""CREATE KEYSPACE peliculas WITH replication = {'class' : 'SimpleStrategy', 'replication_factor':1};""")


### Creación de la Tablas.

In [21]:
# Seleccionamos el KEYSPACE
session.set_keyspace('peliculas')

# Creamos la tabla
session.execute("""
                CREATE TABLE usuarios (
                    membresia int,
                    nombre text,
                    a_p text,
                    a_m text,
                    pais text,
                    fechnac timestamp,
                    id int,
                    titulo text,
                    autor text, 
                    categoria text,
                    calificacion float,
                    PRIMARY KEY (membresia, id));
                """)


In [ ]:
'''
Podemos consultar las tablas creadas con:
DESCRIBE tables
'''